In [130]:
import requests
import re
import datetime as dt
import time
from bs4 import BeautifulSoup
import os
from pandas.io.json import json_normalize
from pymongo import MongoClient
import urllib.request, json
regex = re.compile('offer-path.*')
url='https://www.olx.pl/oferty/q-dreamcast/'

page = requests.get(url)

In [122]:
os.chdir('/Users/miroslawjacoszek/python/allegro')

f = open('config/config.json') 

config_json = json.load(f) 
  
f.close()

In [123]:
mongo_connection_string=config_json['mongodb']['connection_string']

In [106]:
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.findAll("table",{"summary": "Ogłoszenie"})

In [107]:
title=results[1].find("a",{"data-cy":"listing-ad-title"}).strong.get_text()
link =results[1].find('a', href=True)['href']
import re
regex = re.compile('offer-path.*')
category = results[1].find("p", {"class" : regex}).get_text().replace('\n', '').strip()


In [ ]:
new_rows=1
    
while True: 
    
    page = requests.get(url)
    
    current_time = dt.datetime.now()

    if new_rows ==1:
        with MongoClient(mongo_connection_string) as client_mongo:
            mydb= client_mongo["allegro"]
            offer_tbl = mydb["offers"]
            offer_list=list(offer_tbl.find({},{'link':1, "_id": 0}))
            offer_list=[d['link'] for d in offer_list]
            
        new_rows = 0
        
    for r in results:
         
        link =r.find('a', href=True)['href']
        
        if link in offer_list == False:
            
            title=r.find("a",{"data-cy":"listing-ad-title"}).strong.get_text().replace('\n', '')
            link =r.find('a', href=True)['href']
            category = r.find("p", {"class" : regex}).get_text().replace('\n', '').strip()
            price=r.find("p",{"class":"price"}).get_text().replace('\n', '')
            location=results[1].find("i",{"data-icon": "location-filled"}).next_element
            added=results[1].find("i",{"data-icon": "clock"}).next_element
            record_creation_date=current_time.strftime('%Y-%m-%d %H:%M:%S')

            offer= {"link": link,
                    "title": title,
                    "category": category,
                    "price": price,
                    "location": location,
                    "added": added,
                    "record_creation_date": record_creation_date}

            with MongoClient(mongo_connection_string) as client_mongo:
                mydb= client_mongo["allegro"]
                mycol = mydb["offers"]
                mycol.insert_one(offer)
            
            new_rows = 1
            
            print(offer)

    #break
    print('sleep')
    time.sleep(600)
    

sleep
sleep
